# Flight Rerouting Analysis at Dublin Airport

This project investigates flight rerouting events at Dublin Airport and their relationship to local weather conditions. The analysis combines flight activity data with historical and forecast weather data from Met Éireann to identify trends, visualise reroute reasons, and project future rerouting probabilities.

### 📦 Step 2 – Install and Import Required Libraries

This step prepares the environment for the Dublin Airport Flight Rerouting Project.  
It ensures that all required Python packages are available and sets up the project’s directory structure inside the `project` root.

The notebook imports essential libraries for:

- 📊 **Data manipulation** (`pandas`, `numpy`)
- 📅 **Date and time handling** (`datetime`, `matplotlib.dates`)
- 📈 **Plotting and visualisation** (`matplotlib`, `seaborn`, `plotly`)
- 🤖 **Machine learning and model persistence** (`scikit-learn`, `joblib`)
- 📂 **File handling and paths** (`os`, `pathlib`, `json`)
- 🌐 **Web access** (`requests`)
- 🧩 **Interactivity and display** (`ipywidgets`, `IPython.display`)

It also defines key directories (`data`, `outputs`, `models`, `docs`) inside the `project` folder and ensures they exist.  
This structure keeps raw data, processed outputs, trained models, and documentation organised and reproducible.

📌 *Note: `%pip install` commands can be used inside Jupyter notebooks if a package is missing.  
For scripts or terminal use, run `pip install` directly.*


In [1]:
%pip install plotly --quiet

# Setup: imports, paths and basic config
import json  # for any config files
from pathlib import Path  # for path management
import numpy as np  # numerical operations
import pandas as pd  # data manipulation
import matplotlib.pyplot as plt  # plotting
import seaborn as sns  # enhanced plotting
import plotly.express as px  # interactive plotting
import warnings  # to manage warnings

from sklearn.model_selection import train_test_split, cross_val_score  # model validation
from sklearn.linear_model import LogisticRegression  # example model
from sklearn.ensemble import GradientBoostingClassifier  # example model
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix  # model evaluation
import joblib  # model persistence

# Plotting style
sns.set(style='whitegrid')

# Explicit project root: programming-for-data-analytics/project
ROOT = Path.cwd().resolve()
if ROOT.name != "project":
    # climb up until we find project folder
    for parent in ROOT.parents:
        if parent.name == "project":
            ROOT = parent
            break

# Define key directories inside project
DATA_DIR = ROOT / "data"
OUTPUT_DIR = ROOT / "outputs"
MODEL_DIR = ROOT / "models"
DOCS_DIR = ROOT / "docs"

# Ensure directories exist
for path in [DATA_DIR, OUTPUT_DIR, MODEL_DIR, DOCS_DIR]:
    path.mkdir(parents=True, exist_ok=True)

print(f"Project root: {ROOT}")
print(f"Data directory: {DATA_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Model directory: {MODEL_DIR}")
print(f"Docs directory: {DOCS_DIR}")


Note: you may need to restart the kernel to use updated packages.
Project root: C:\Users\eCron\OneDrive\Documents\ATU_CourseWork\Programming For Data Analytics\programming-for-data-analytics\project
Data directory: C:\Users\eCron\OneDrive\Documents\ATU_CourseWork\Programming For Data Analytics\programming-for-data-analytics\project\data
Output directory: C:\Users\eCron\OneDrive\Documents\ATU_CourseWork\Programming For Data Analytics\programming-for-data-analytics\project\outputs
Model directory: C:\Users\eCron\OneDrive\Documents\ATU_CourseWork\Programming For Data Analytics\programming-for-data-analytics\project\models
Docs directory: C:\Users\eCron\OneDrive\Documents\ATU_CourseWork\Programming For Data Analytics\programming-for-data-analytics\project\docs


### Step 3 – Utilise Helper Functions for Dublin Airport Data Processing

This section defines a set of reusable helper functions that simplify common tasks in the project.  
They are designed specifically to support the analysis of **Dublin Airport flight activity and weather data** by handling messy inputs and preparing clean datasets for exploration and modelling.

The functions help with:

- ✅ Detecting and parsing inconsistent datetime formats in flight and weather logs  
- ✅ Standardising and cleaning temperature columns from Met Éireann datasets  
- ✅ Loading and preparing Dublin Airport daily weather data from local CSV files  
- ✅ Defining Irish seasonal boundaries for rerouting analysis (Winter, Spring, Summer, Autumn)  
- ✅ Filtering weather data for a custom date range to align with flight events  
- ✅ Validating user-provided date inputs for reproducible analysis  
- ✅ Detecting header rows in raw CSV files downloaded from dashboards  

Each helper is **modular** — it performs one clear task and can be reused across notebooks and scripts.  
This improves readability, reduces duplication, and supports good programming practices for the final project.

📌 *Tip: These helpers are written to be beginner-friendly, with comments explaining their purpose and logic. They make it easier to align flight activity with weather conditions when investigating rerouting events.*

📖 References:  
- [Real Python – Python Helper Functions](https://realpython.com/defining-your-own-python-function/)  
- [GeeksforGeeks – Python Helper Functions](https://www.geeksforgeeks.org/python-helper-functions/)  
- [Wikipedia – DRY Principle (Don't Repeat Yourself)](https://en.wikipedia.org/wiki/Don%27t_repeat_yourself)


In [2]:
# 📂 Helper Functions for Dublin Airport Project
# These functions help with parsing dates, cleaning weather data, handling temperature columns,
# defining Irish seasons, preparing data ranges, and detecting CSV headers.
# Keep them in one cell so they are easy to reuse across the notebook.

# important imports already loaded above
"""
import pandas as pd
import warnings
from pathlib import Path
"""

# 📅 Detect the most likely datetime format from sample strings
def detect_datetime_format(samples, formats, dayfirst=True):
    """
    Try each format and return the one that matches at least 70% of samples.
    Helps ensure consistent parsing of date strings.
    """
    for fmt in formats:
        parsed = pd.to_datetime(samples, format=fmt, dayfirst=dayfirst, errors='coerce')
        if parsed.notna().sum() >= max(1, int(len(samples) * 0.7)):
            return fmt
    return None

# 📅 Parse a datetime column using format detection or fallback
def parse_datetime_column(df, date_col, candidate_formats=None, dayfirst=True):
    """
    Parse a datetime column using known formats.
    Falls back to flexible parsing if none match.
    """
    if candidate_formats is None:
        candidate_formats = [
            '%Y-%m-%d %H:%M:%S', '%Y-%m-%d %H:%M', '%d-%b-%Y %H:%M',
            '%d/%m/%Y %H:%M:%S', '%d/%m/%Y %H:%M', '%d-%m-%Y %H:%M',
            '%d %b %Y %H:%M', '%d %B %Y %H:%M',
        ]

    sample_vals = df[date_col].dropna().astype(str).head(80).tolist()
    chosen_fmt = detect_datetime_format(sample_vals, candidate_formats, dayfirst=dayfirst)

    if chosen_fmt:
        print(f"✅ Detected datetime format: {chosen_fmt}")
        return pd.to_datetime(df[date_col], format=chosen_fmt, dayfirst=dayfirst, errors='coerce')
    else:
        print("⚠️ No single format matched. Falling back to flexible parsing.")
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', message='Could not infer format')
            return pd.to_datetime(df[date_col], dayfirst=dayfirst, errors='coerce')

# 🌡️ Ensure temperature column is numeric and named 'temp'
def parse_temperature_column(df, col_name='temp'):
    """
    Convert the temperature column to numeric and rename it to 'temp'.
    If no exact match, look for any column containing 'temp'.
    """
    if col_name not in df.columns:
        col_name = next((c for c in df.columns if 'temp' in c.lower()), None)
        if col_name is None:
            raise KeyError("No temperature column found.")
    df['temp'] = pd.to_numeric(df[col_name], errors='coerce')
    return df

# 📂 Load cleaned weather data from local CSV
def load_cleaned_weather_data(filepath="data/dublin_airport_daily.csv"):
    """
    Load weather dataset from CSV and strip spaces from column names.
    """
    df = pd.read_csv(filepath, low_memory=False)
    df.columns = df.columns.str.strip()
    return df

# 🍂 Define Irish seasonal boundaries for a given year
def define_irish_seasons(year=2025):
    """
    Return start and end dates for Irish meteorological seasons.
    """
    data = [
        ("Winter", pd.Timestamp(f"{year-1}-12-01"), pd.Timestamp(f"{year}-02-28 23:59")),
        ("Spring", pd.Timestamp(f"{year}-03-01"), pd.Timestamp(f"{year}-05-31 23:59")),
        ("Summer", pd.Timestamp(f"{year}-06-01"), pd.Timestamp(f"{year}-08-31 23:59")),
        ("Autumn", pd.Timestamp(f"{year}-09-01"), pd.Timestamp(f"{year}-11-30 23:59")),
    ]
    return pd.DataFrame(data, columns=["season", "start", "end"])

# 📊 Filter and prepare temperature data for a custom date range
def prepare_temperature_data(df, start_date, end_date):
    """
    Filter weather data to a date range and add useful time features.
    """
    df = df.copy()
    df.columns = df.columns.str.strip()

    if 'date' not in df.columns:
        raise KeyError("Expected 'date' column not found.")

    # Try parsing with a common format, fallback to flexible parsing
    try:
        df['datetime'] = pd.to_datetime(df['date'], format='%d-%b-%Y %H:%M', errors='raise')
    except Exception:
        df['datetime'] = pd.to_datetime(df['date'], errors='coerce')

    df = df.dropna(subset=['datetime'])
    mask = (df['datetime'] >= start_date) & (df['datetime'] <= end_date)
    range_df = df.loc[mask].copy()

    # Add date and hour columns for plotting
    range_df['date'] = range_df['datetime'].dt.date
    range_df['hour'] = range_df['datetime'].dt.strftime('%H:%M')

    return range_df.sort_values('datetime').reset_index(drop=True)

# 📆 Convert user input strings into a validated date range
def get_custom_range(start_str, end_str):
    """
    Convert string inputs into datetime objects and validate order.
    """
    try:
        start = pd.to_datetime(start_str)
        end = pd.to_datetime(end_str)
        if start > end:
            raise ValueError("Start date must be before end date.")
        return start, end
    except Exception as e:
        print(f"❌ Invalid date range: {e}")
        return None, None

# 🔍 Detect the header row in a CSV file
def detect_header(lines):
    """
    Detect the most likely header row in a CSV file.
    Looks for lines starting with 'date' or 'station' and containing commas.
    """
    for i, line in enumerate(lines):
        line_lower = line.strip().lower()
        if (line_lower.startswith("station") or line_lower.startswith("date")) and "," in line:
            columns = line.split(",")
            if len(columns) > 5:  # Header rows usually have multiple columns
                return i
    print("⚠️ Warning: header row not found. Defaulting to first line.")
    return 0


### 📂 Step 4 – Download Dublin Airport Daily Data and Detect Header Row

In this step, the notebook retrieves the **Dublin Airport Daily Data CSV** directly from Met Éireann’s open data service.  
This dataset contains daily weather observations (e.g., precipitation, temperature, wind speed, radiation) recorded at Dublin Airport, which will later be aligned with flight activity logs to analyse rerouting events.

The process includes:

- 🌐 **Downloading the raw CSV** from Met Éireann using the `requests` library.  
- 📂 **Defining a local output path** (`data/dublin_airport_daily.csv`) to store the file inside the project’s `data` folder.  
- ✅ **Checking the HTTP response** to ensure the download was successful.  
- 📑 **Splitting the file into lines** so the structure can be inspected before loading into pandas.  
- 🔍 **Detecting the header row** using the `detect_header` helper function defined earlier.  
  This ensures that column names (such as `date`, `maxtp`, `mintp`, `rain`, `wdsp`) are correctly identified even if the file contains metadata lines at the top.  
- 🖨️ **Printing the detected header row** to confirm the correct starting point for parsing.

📌 *Tip: Detecting the header row is important because Met Éireann CSVs often include metadata lines before the actual data table.  
By confirming the header row, you avoid misaligned columns and ensure clean parsing in later steps.*


In [3]:
# 📂 Step 4 – Download Dublin Airport Daily Data CSV and Detect Header Row

from pathlib import Path
import requests

# --- Define output path for cleaned CSV ---
DATA_PATH = Path("data/dublin_airport_daily.csv")

# --- Download raw CSV from Met Éireann (Dublin Airport Daily Data) ---
url = "https://cli.fusio.net/cli/climate_data/webdata/dly532.csv"
response = requests.get(url)

# ✅ Check for successful response
if response.status_code != 200:
    raise RuntimeError(f"❌ Failed to download data: HTTP {response.status_code}")

# --- Split response into lines ---
lines = response.text.splitlines()

# --- Detect header row using helper function ---
header_index = detect_header(lines)

# ✅ Confirm detected header row
print(f"✅ Header row detected at line {header_index}:")
print(lines[header_index])


✅ Header row detected at line 25:
date,ind,maxtp,ind,mintp,igmin,gmin,ind,rain,cbl,wdsp,ind,hm,ind,ddhm,ind,hg,sun,dos,g_rad,soil,pe,evap,smd_wd,smd_md,smd_pd


### 📑 Step 4b – Load and Inspect Dublin Airport Daily Data

After detecting the correct header row in the raw CSV file, the next step is to **load the dataset into pandas**.  
This allows us to immediately inspect the structure of the Dublin Airport Daily Data and confirm that the columns (e.g., `date`, `maxtp`, `mintp`, `rain`, `wdsp`) are correctly aligned.

The process includes:

- 📂 Reading the CSV into a pandas DataFrame, starting from the detected header row  
- 🔍 Displaying the first few rows with `head()` to verify column names and sample values  
- 🧾 Using `info()` to check datatypes and identify potential missing values  
- 📊 Summarising numeric columns with `describe()` to get a quick statistical overview  

📌 *Why this matters:* Inspecting the dataset before saving ensures that the header detection worked correctly and that the file is ready for consistent downstream analysis.  
This step acts as a validation checkpoint before committing the cleaned file to the `data/` folder.


In [4]:
# 📑 Step 4b – Load and Inspect Dublin Airport Daily Data

# --- Load CSV into pandas using detected header row ---
df = pd.read_csv(
    url,  # still using the online source
    skiprows=header_index  # skip metadata lines before the header
)

# ✅ Inspect the first few rows
print("First 5 rows of Dublin Airport Daily Data:")
display(df.head())

# ✅ Check column types and missing values
print("\nDataFrame info:")
print(df.info())

# ✅ Quick statistical summary
print("\nSummary statistics:")
print(df.describe(include='all'))


First 5 rows of Dublin Airport Daily Data:


,date,ind,maxtp,ind.1,mintp,igmin,gmin,ind.2,rain,cbl,...,hg,sun,dos,g_rad,soil,pe,evap,smd_wd,smd_md,smd_pd
0,01-jan-1942,0,9.7,0,6.8,0,4.7,2,0.0,1020.3,...,,0.0,0,,,1.1,1.4,,,
1,02-jan-1942,0,9.9,0,7.9,0,6.7,0,0.1,1016.2,...,,0.0,0,,,0.7,0.9,,,
2,03-jan-1942,0,11.2,0,8.9,0,7.2,0,1.5,1006.8,...,,0.1,0,,,0.5,0.6,,,
3,04-jan-1942,0,9.2,0,2.7,0,3.4,0,3.5,1001.5,...,,0.6,0,,,0.6,0.7,,,
4,05-jan-1942,0,3.5,1,-0.8,0,0.0,0,0.6,1013.4,...,,3.4,0,,,0.6,0.7,,,



DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30620 entries, 0 to 30619
Data columns (total 26 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    30620 non-null  object 
 1   ind     30620 non-null  int64  
 2   maxtp   30620 non-null  float64
 3   ind.1   30620 non-null  int64  
 4   mintp   30620 non-null  float64
 5   igmin   30620 non-null  int64  
 6   gmin    30620 non-null  object 
 7   ind.2   30620 non-null  int64  
 8   rain    30620 non-null  float64
 9   cbl     30620 non-null  float64
 10  wdsp    30620 non-null  float64
 11  ind.3   30620 non-null  int64  
 12  hm      30620 non-null  object 
 13  ind.4   30620 non-null  int64  
 14  ddhm    30620 non-null  object 
 15  ind.5   30620 non-null  int64  
 16  hg      30620 non-null  object 
 17  sun     30620 non-null  float64
 18  dos     30620 non-null  object 
 19  g_rad   30620 non-null  object 
 20  soil    30620 non-null  object 
 21  pe      30620 non-

### 📁 Step 5 – Save the Cleaned Dublin Airport Daily Data CSV

After detecting the correct header row in the raw Met Éireann dataset, we now save a **cleaned version** of the Dublin Airport Daily Data file into the project’s `data/` folder.  

This step ensures:

- 📂 The dataset is stored locally for reuse without needing to re-download from Met Éireann each time  
- 📑 All future analysis references a consistent, structured version of the data (starting at the correct header row)  
- 🔄 The workflow remains reproducible and version-controlled, supporting transparent project documentation  
- 🛠️ Analysts and reviewers can always work from the same baseline dataset, avoiding inconsistencies caused by raw file metadata  

📌 *Why this matters:*  
Saving cleaned data locally is a best practice in data science. It guarantees consistency across runs, makes collaboration easier, and allows you to track changes over time.  
This approach supports reproducibility and version control in your Dublin Airport rerouting analysis.  

📖 Reference:  
- [GeeksforGeeks – Explain Data Versioning](https://www.geeksforgeeks.org/machine-learning/explain-data-versioning/)


In [5]:
# 📁 Step 5 – Save the Cleaned CSV File

# --- Ensure 'data' folder exists ---
DATA_PATH.parent.mkdir(parents=True, exist_ok=True)

# --- Save cleaned data starting from the detected header row ---
with open(DATA_PATH, "w", encoding="utf-8") as f:
    for line in lines[header_index:]:
        f.write(line + "\n")

# ✅ Confirm save location
print(f"📁 Saved cleaned climate data for Dublin Airport to: {DATA_PATH.resolve()}")


📁 Saved cleaned climate data for Dublin Airport to: C:\Users\eCron\OneDrive\Documents\ATU_CourseWork\Programming For Data Analytics\programming-for-data-analytics\project\data\dublin_airport_daily.csv


### 📂 Step 6 – Validate Saved CSV Against Step 4 Output

Instead of re‑printing the same inspection results, this step **confirms that the locally saved CSV file is identical to the dataset inspected in Step 4**.  

The process includes:

- 📂 Reloading the locally saved CSV (`data/dublin_airport_daily.csv`)  
- 🌐 Reloading the online CSV directly from Met Éireann (skipping metadata lines)  
- ✅ Comparing the two DataFrames with `equals()` to check for exact match  
- 📊 Printing a simple confirmation message and shape comparison  

📌 *Why this matters:* This validation ensures reproducibility. It proves that the cleaned file saved in Step 5 is a faithful copy of the dataset originally inspected in Step 4.  
Reviewers can trust that all downstream analysis is based on the same consistent dataset.

[https://www.geeksforgeeks.org/create-effective-and-reproducible-code-using-pandas/](https://www.geeksforgeeks.org/create-effective-and-reproducible-code-using-pandas/)

In [7]:
# 📂 Step 6 – Validate Saved CSV Against Step 4 Output

import pandas as pd

# --- Reload the locally saved CSV ---
df_local = pd.read_csv(DATA_PATH)

# --- Reload the online CSV (using header_index from Step 4) ---
df_online = pd.read_csv(url, skiprows=header_index)

# ✅ Compare the two DataFrames
if df_local.equals(df_online):
    print("✅ Validation successful: Local CSV matches the online dataset from Step 4.")
else:
    print("❌ Validation failed: Local CSV differs from the online dataset.")

# Optional: show shape comparison
print(f"Local shape: {df_local.shape}, Online shape: {df_online.shape}")


✅ Validation successful: Local CSV matches the online dataset from Step 4.
Local shape: (30620, 26), Online shape: (30620, 26)


### 📑 Step 7 – Download and Save Flight Activity Data

In this step, the notebook retrieves and stores **flight activity data** for Dublin Airport.  
This dataset will later be aligned with weather observations to analyse rerouting events.

The process includes:

- 🌐 Collecting flight activity logs from Dublin Airport dashboards or trackers (arrivals, departures, runway usage)  
- 📂 Defining a local output path (`data/dublin_airport_flights.csv`) to store the file inside the project’s `data` folder  
- ✅ Checking the response to ensure the download or export was successful  
- 📑 Splitting the file into lines and detecting the header row (similar to weather data)  
- 📁 Saving a cleaned version of the dataset locally for reproducibility  

Additionally, you will begin constructing an **annotated reroute CSV** (`data/reroutes.csv`) where you log rerouting events manually or semi‑automatically.  
This file should include:  
- `date`, `time`, `flight_id`, `runway_before`, `runway_after`, `reason`  

📌 *Why this matters:* Having both flight activity logs and reroute annotations stored locally ensures that your analysis can consistently align flight events with weather conditions.  
It also supports reproducibility and version control across the project.
